In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_file=r"D:\文件\学习\数据集\二手车交易\used_car_train_20200313\used_car_train_20200313.csv"
test_file=r"D:\文件\学习\数据集\二手车交易\used_car_testB_20200421\used_car_testB_20200421.csv"

In [3]:
pd.set_option('display.max_columns',100)
pd.set_option('expand_frame_repr',False)

In [4]:
train_df = pd.read_csv(train_file, sep=" ")
test_df = pd.read_csv(test_file, sep=" ")

## 建立自定义方法

In [5]:
class car_method:
    def __init__(self, dataset):
        self.dataset = dataset
    #众数填充缺失值的函数
    def fill_with_mode(self, column_name):
        mode_ = self.dataset[column_name].value_counts().index[0]
        self.dataset[column_name] = self.dataset[column_name].fillna(mode_)
        return self.dataset
    #用哑变量处理分类数据的函数
    def get_dummies_column(self, column_name):
        dummy_ = pd.get_dummies(self.dataset[column_name], prefix=column_name)
        self.dataset = pd.concat([self.dataset, dummy_], axis = 1)
        self.dataset = self.dataset.drop(column_name, axis =1)
        return self.dataset
    #数据分箱函数
    def cut_column(self, column_name, k=5):
        cut_ = pd.cut(x=self.dataset[column_name], bins=k, labels=range(1,k+1))
        cut_ = pd.DataFrame(cut_)
        cut_.columns=[column_name+'_{}'.format('cut')]
        self.dataset = pd.concat([self.dataset, cut_], axis = 1)
        return self.dataset

## 数据清洗

In [6]:
#用众数填充缺失值

In [7]:
fill_with_mode =car_method(train_df)
train_df = fill_with_mode.fill_with_mode('model')
test_df = fill_with_mode.fill_with_mode('model')

train_df = fill_with_mode.fill_with_mode('bodyType')
test_df = fill_with_mode.fill_with_mode('bodyType')

train_df = fill_with_mode.fill_with_mode('fuelType')
test_df = fill_with_mode.fill_with_mode('fuelType')

train_df = fill_with_mode.fill_with_mode('gearbox')
test_df = fill_with_mode.fill_with_mode('gearbox')

In [8]:
train_df['notRepairedDamage'] = train_df['notRepairedDamage'].replace('-','0.0')
test_df['notRepairedDamage'] = test_df['notRepairedDamage'].replace('-','0.0')

In [9]:
#用哑变量处理分类数据
#notRepairedDamage
get_dummies_ = car_method(train_df)
train_df = get_dummies_.get_dummies_column('notRepairedDamage')

get_dummies02 = car_method(test_df)
test_df =get_dummies02.get_dummies_column('notRepairedDamage')

In [10]:
#对kilometer进行数据分箱
cut_ = car_method(train_df)
train_df = cut_.cut_column('kilometer', 5)

In [11]:
#对v_0-v_14进行二值化
for i in range(1,15):
    from sklearn.preprocessing import Binarizer
    n = np.min(train_df['v_{}'.format(i)])+(np.max(train_df['v_{}'.format(i)])-np.min(train_df['v_{}'.format(i)]))/2
    bin_ = Binarizer(threshold=n).fit_transform(train_df['v_{}'.format(i)].values.reshape(-1,1)).reshape(1,-1)[0]
    bin_ = pd.DataFrame(bin_)
    bin_.columns=['v_{}{}'.format(i,'_bin')]
    train_df = pd.concat([train_df, bin_], axis =1)

In [12]:
train_df.describe()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,notRepairedDamage_0.0,notRepairedDamage_1.0,v_1_bin,v_2_bin,v_3_bin,v_4_bin,v_5_bin,v_6_bin,v_7_bin,v_8_bin,v_9_bin,v_10_bin,v_11_bin,v_12_bin,v_13_bin,v_14_bin
count,150000.000000,150000.000000,1.500000e+05,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.0,1.500000e+05,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,74999.500000,68349.172873,2.003417e+07,47.128707,8.052733,1.738527,0.354093,0.215973,119.316547,12.597160,2583.077267,0.000007,0.0,2.016033e+07,5923.327333,44.406268,-0.044809,0.080765,0.078833,0.017875,0.248204,0.044923,0.124692,0.058144,0.061996,-0.001000,0.009035,0.004813,0.000313,-0.000688,0.904567,0.095433,0.420420,0.030040,0.299247,0.159760,0.969960,0.406567,0.030040,0.239587,0.088293,0.503747,0.030040,0.210420,0.002893,0.115933
std,43301.414527,61103.875095,5.364988e+04,49.536024,7.864956,1.760778,0.539747,0.411497,177.168419,3.919576,1885.363218,0.002582,0.0,1.067328e+02,7501.998477,2.457548,3.641893,2.929618,2.026514,1.193661,0.045804,0.051743,0.201410,0.029186,0.035692,3.772386,3.286071,2.517478,1.288988,1.038685,0.293814,0.293814,0.493628,0.170698,0.457930,0.366385,0.170698,0.491194,0.170698,0.426833,0.283722,0.499988,0.170698,0.407608,0.053712,0.320146
min,0.000000,0.000000,1.991000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,2.015062e+07,11.000000,30.451976,-4.295589,-4.470671,-7.275037,-4.364565,0.000000,0.000000,0.000000,0.000000,0.000000,-9.168192,-5.558207,-9.639552,-4.153899,-6.546556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37499.750000,11156.000000,1.999091e+07,10.000000,1.000000,0.000000,0.000000,0.000000,75.000000,12.500000,1018.000000,0.000000,0.0,2.016031e+07,1300.000000,43.135799,-3.192349,-0.970671,-1.462580,-0.921191,0.243615,0.000038,0.062474,0.035334,0.033930,-3.722303,-1.951543,-1.871846,-1.057789,-0.437034,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,74999.500000,51638.000000,2.003091e+07,30.000000,6.000000,1.000000,0.000000,0.000000,110.000000,15.000000,2196.000000,0.000000,0.0,2.016032e+07,3250.000000,44.610266,-3.052671,-0.382947,0.099722,-0.075910,0.257798,0.000812,0.095866,0.057014,0.058484,1.624076,-0.358053,-0.130753,-0.036245,0.141246,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,112499.250000,118841.250000,2.007111e+07,66.000000,13.000000,3.000000,1.000000,0.000000,150.000000,15.000000,3843.000000,0.000000,0.0,2.016033e+07,7700.000000,46.004721,4.000670,0.241335,1.565838,0.868758,0.265297,0.102009,0.125243,0.079382,0.087491,2.844357,1.255022,1.776933,0.942813,0.680378,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,149999.000000,196812.000000,2.015121e+07,247.000000,39.000000,7.000000,6.000000,1.000000,19312.000000,15.000000,8120.000000,1.000000,0.0,2.016041e+07,99999.000000,52.304178,7.320308,19.035496,9.854702,6.829352,0.291838,0.151420,1.404936,0.160791,0.222787,12.357011,18.819042,13.847792,11.147669,8.658418,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000

## 数据概况

In [17]:
train_df.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,notRepairedDamage_0.0,notRepairedDamage_1.0,kilometer_cut,v_1_bin,v_2_bin,v_3_bin,v_4_bin,v_5_bin,v_6_bin,v_7_bin,v_8_bin,v_9_bin,v_10_bin,v_11_bin,v_12_bin,v_13_bin,v_14_bin
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,1046,0,0,20160404,1850,43.357796,3.966344,0.050257,2.159744,1.143786,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762,1,0,5,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,4366,0,0,20160309,3600,45.305273,5.236112,0.137925,1.380657,-1.422165,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522,1,0,5,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,2806,0,0,20160402,6222,45.978359,4.823792,1.319524,-0.998467,-0.996911,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963,1,0,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,434,0,0,20160312,2400,45.687478,4.492574,-0.050616,0.883600,-2.228079,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699,1,0,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,6977,0,0,20160313,5200,44.383511,2.031433,0.572169,-1.571239,2.246088,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482,1,0,2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   SaleID                 150000 non-null  int64   
 1   name                   150000 non-null  int64   
 2   regDate                150000 non-null  int64   
 3   model                  150000 non-null  float64 
 4   brand                  150000 non-null  int64   
 5   bodyType               150000 non-null  float64 
 6   fuelType               150000 non-null  float64 
 7   gearbox                150000 non-null  float64 
 8   power                  150000 non-null  int64   
 9   kilometer              150000 non-null  float64 
 10  regionCode             150000 non-null  int64   
 11  seller                 150000 non-null  int64   
 12  offerType              150000 non-null  int64   
 13  creatDate              150000 non-null  int64   
 14  price               

In [15]:
test_df.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,regionCode,seller,offerType,creatDate,price,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,notRepairedDamage_0.0,notRepairedDamage_1.0
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,1046,0,0,20160404,1850,43.357796,3.966344,0.050257,2.159744,1.143786,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762,1,0
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,4366,0,0,20160309,3600,45.305273,5.236112,0.137925,1.380657,-1.422165,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522,1,0
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,2806,0,0,20160402,6222,45.978359,4.823792,1.319524,-0.998467,-0.996911,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963,1,0
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,434,0,0,20160312,2400,45.687478,4.492574,-0.050616,0.883600,-2.228079,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699,1,0
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,6977,0,0,20160313,5200,44.383511,2.031433,0.572169,-1.571239,2.246088,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482,1,0


In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   SaleID                 150000 non-null  int64  
 1   name                   150000 non-null  int64  
 2   regDate                150000 non-null  int64  
 3   model                  150000 non-null  float64
 4   brand                  150000 non-null  int64  
 5   bodyType               150000 non-null  float64
 6   fuelType               150000 non-null  float64
 7   gearbox                150000 non-null  float64
 8   power                  150000 non-null  int64  
 9   kilometer              150000 non-null  float64
 10  regionCode             150000 non-null  int64  
 11  seller                 150000 non-null  int64  
 12  offerType              150000 non-null  int64  
 13  creatDate              150000 non-null  int64  
 14  price                  150000 non-nu